In [156]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [157]:
import pandas as pd
import os
import shutil

from sklearn.model_selection import train_test_split

In [158]:
import json

In [159]:
votes = pd.read_csv('/home/walml/repos/gz-panoptes-reduction/data/jwst/jwst-ceers-v0-5-streamlined-clumpy-classifications.csv')
votes = votes.rename(columns={'subject_ids': 'subject_id'})

aggregated = pd.read_csv('/home/walml/repos/gz-panoptes-reduction/data/jwst/jwst-ceers-v0-5_reduced_votes.csv')
image_root = '/home/walml/repos/galaxy-datasets/derived_data/jwst'

In [160]:
subject_loc = '/home/walml/Downloads/galaxy-zoo-subjects.csv'
subject_id_vs_url = pd.read_csv(subject_loc, nrows=None, usecols=['subject_id', 'subject_set_id', 'locations']).query('subject_set_id == 112884').drop_duplicates()

In [161]:
subject_id_vs_url['url'] = subject_id_vs_url['locations'].apply(lambda x: list(json.loads(x).values())[0])
subject_id_vs_url['url'].iloc[0]

'https://panoptes-uploads.zooniverse.org/subject_location/a73c2d24-ec2f-47fa-accd-d41dab5d640f.png'

In [162]:
votes = votes.drop_duplicates(subset=['subject_id']).reset_index(drop=True)

In [163]:
# some kung fu pandas to extract the subject metadata as a new df
subject_df = votes[['subject_id']]
subject_metadata = pd.DataFrame(
    votes['subject_data'].apply(lambda x: list(json.loads(x).values())[0]).tolist()
)
# will align by index
subject_df = subject_df.join(subject_metadata)

subject_df = pd.merge(subject_df, subject_id_vs_url[['subject_id', 'url']], on='subject_id', how='inner', validate='1:1')


In [164]:

# subject_df['subfolder'] = subject_df['subject_id'].apply(lambda x: x[:21])
# subject_df['file_exists']
# subject_df['subfolder'].value_counts()

In [165]:
# subject_df['file_loc'] = image_root + '/' + subject_df['subfolder'] + '/' + subject_df['subject_id'] + '.jpg'
# subject_df['file_loc'].iloc[0]

In [166]:
# pull all of these with wget
# with open(image_root + '/latest_image_urls.txt', 'w') as f:
    # f.write('\n'.join(subject_df['url'].values))

In [167]:
aggregated

,id_str,smooth-or-featured_smooth,smooth-or-featured_featured-or-disk,smooth-or-featured_star-artifact-zoom,how-rounded_round,how-rounded_in-between,how-rounded_cigar-shaped,disk-edge-on_yes,disk-edge-on_no,edge-on-bulge_rounded,...,bulge-size_dominant_fraction,merging_merger_fraction,merging_major-disturbance_fraction,merging_minor-disturbance_fraction,merging_none_fraction,clumps_yes_fraction,clumps_no_fraction,problem_star_fraction,problem_artifact_fraction,problem_bad-zoom_fraction
0,10000,0,30,2,0,0,0,0,30,0,...,0.1,0.000000,0.100000,0.366667,0.533333,0.666667,0.333333,0.000000,0.000000,1.000000
1,10001,24,2,11,2,21,1,0,2,0,...,0.0,0.000000,0.038462,0.192308,0.769231,0.000000,1.000000,0.181818,0.000000,0.818182
2,10011,7,8,13,5,2,0,0,8,0,...,0.0,0.200000,0.133333,0.133333,0.533333,0.625000,0.375000,0.153846,0.153846,0.692308
3,10015,5,0,31,1,3,1,0,0,0,...,0.0,0.000000,0.200000,0.600000,0.200000,0.000000,0.000000,0.000000,0.225806,0.774194
4,10016,2,0,22,0,2,0,0,0,0,...,0.0,0.000000,0.000000,0.500000,0.500000,0.000000,0.000000,0.000000,0.136364,0.863636
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7674,9992,6,11,7,0,3,3,2,9,1,...,0.0,0.352941,0.117647,0.235294,0.294118,0.636364,0.363636,0.000000,0.428571,0.571429
7675,9993,5,13,5,0,5,0,1,12,0,...,0.0,0.277778,0.444444,0.055556,0.222222,0.692308,0.307692,0.000000,0.400000,0.600000
7676,9995,10,7,5,1,9,0,5,2,4,...,0.0,0.058824,0.000000,0.117647,0.823529,0.000000,1.000000,0.200000,0.200000,0.600000
7677,9996,13,6,7,1,10,2,1,5,0,...,0.2,0.105263,0.052632,0.210526,0.631579,0.166667,0.833333,0.000000,0.000000,1.000000


In [168]:
subject_df = subject_df.rename(columns={'id': 'id_str'})
subject_df['id_str'] = subject_df['id_str'].astype(int)

In [169]:
df = pd.merge(subject_df, aggregated)
df

,subject_id,retired,RA,id_str,Dec,image,brtfac,pixrad,nircam_id,mag_select,...,bulge-size_dominant_fraction,merging_merger_fraction,merging_major-disturbance_fraction,merging_minor-disturbance_fraction,merging_none_fraction,clumps_yes_fraction,clumps_no_fraction,problem_star_fraction,problem_artifact_fraction,problem_bad-zoom_fraction
0,88112645,None,214.902789,14717,52.877670,g5622_nircam2_ceers.png,1.2486870588753547,81,5622,22.9,...,0.00,0.000000,0.000000,0.000000,1.000000,0.00,0.00,0.0,0.242424,0.757576
1,88115594,None,214.836407,24116,52.886848,g4729_nircam3_ceers.png,1.25,107,4729,23.67,...,0.00,0.125000,0.000000,0.312500,0.562500,1.00,0.00,0.0,0.000000,1.000000
2,88111551,None,214.905918,11751,52.928909,g2656_nircam2_ceers.png,1.2491184018463928,73,2656,22.93,...,0.10,0.038462,0.423077,0.153846,0.384615,0.20,0.80,0.0,0.230769,0.769231
3,88106122,None,214.985558,4964,53.002966,g4132_nircam1_ceers.png,1.25,105,4132,23.18,...,0.00,0.105263,0.368421,0.263158,0.263158,0.00,1.00,0.0,0.000000,1.000000
4,88107372,None,214.959103,6977,52.956733,g2119_nircam1_ceers.png,1.25,93,2119,23.89,...,0.00,0.045455,0.136364,0.181818,0.636364,0.20,0.80,0.0,0.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7674,88114238,None,214.773120,21662,52.832526,g2275_nircam3_ceers.png,1.2246345962737069,83,2275,21.99,...,0.00,0.333333,0.166667,0.333333,0.166667,0.00,1.00,0.0,0.000000,1.000000
7675,88106024,None,214.981115,4813,53.002175,g4283_nircam1_ceers.png,1.2256155965486073,69,4283,22.02,...,0.25,0.230769,0.153846,0.307692,0.307692,1.00,0.00,0.0,0.000000,1.000000
7676,88111553,None,214.900195,11758,52.925090,g2663_nircam2_ceers.png,1.25,74,2663,23.33,...,0.00,0.000000,0.000000,0.166667,0.833333,1.00,0.00,0.0,0.125000,0.875000
7677,88105912,None,215.019722,4513,52.991150,g4583_nircam1_ceers.png,1.25,70,4583,23.47,...,0.00,0.153846,0.076923,0.153846,0.615385,0.00,1.00,0.0,0.083333,0.916667


In [170]:
df['raw_file_loc'] = '/home/walml/repos/galaxy-datasets/derived_data/jwst/' + df['url'].apply(os.path.basename)

In [171]:
df['file_loc'] = '/home/walml/repos/galaxy-datasets/derived_data/jwst_final/' + df['subject_id'].astype(str) + '.jpg'

In [172]:
df['raw_file_loc'].iloc[0]

'/home/walml/repos/galaxy-datasets/derived_data/jwst/b5af283f-f0ac-4a51-bf41-cf8f16cef05a.png'

In [182]:
df['file_exists'] = df['raw_file_loc'].apply(os.path.isfile)
df['file_exists'].mean()

1.0

In [184]:
# df[~df['file_exists']]['url'].iloc[0]

In [185]:
from galaxy_datasets.shared import label_metadata

label_cols = label_metadata.jwst_ortho_label_cols

# label_metadata.gz_ukidss_ortho_dependencies

In [186]:
df = df.rename(columns=dict(zip([col.replace('-jwst', '') for col in label_cols], label_cols)))
df = df.rename(columns=dict(zip([col.replace('-jwst', '') + '_fraction' for col in label_cols], [col + '_fraction' for col in label_cols])))

for col in df.columns.values:
    if 'total-votes' in col:
        del df[col]

In [187]:
# df.columns.values

In [188]:
df[label_cols]

,smooth-or-featured-jwst_smooth,smooth-or-featured-jwst_featured-or-disk,smooth-or-featured-jwst_star-artifact-zoom,disk-edge-on-jwst_yes,disk-edge-on-jwst_no,has-spiral-arms-jwst_yes,has-spiral-arms-jwst_no,bar-jwst_strong,bar-jwst_weak,bar-jwst_no,...,spiral-arm-count-jwst_cant-tell,clumps-jwst_yes,clumps-jwst_no,merging-jwst_none,merging-jwst_minor-disturbance,merging-jwst_major-disturbance,merging-jwst_merger,problem-jwst_star,problem-jwst_artifact,problem-jwst_bad-zoom
0,1,0,33,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,8,25
1,15,1,8,0,1,1,0,0,0,1,...,0,1,0,9,5,0,2,0,0,8
2,16,10,13,0,10,1,9,0,1,9,...,0,2,8,10,4,11,1,0,3,10
3,15,4,12,1,3,0,3,0,0,3,...,0,0,4,5,5,7,2,0,0,12
4,17,5,13,1,4,1,3,1,1,2,...,0,1,4,14,4,3,1,0,0,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7674,17,1,5,1,0,0,0,0,0,0,...,0,0,1,3,6,3,6,0,0,5
7675,9,4,6,0,4,0,4,1,1,2,...,0,4,0,4,4,2,3,0,0,6
7676,11,1,8,0,1,0,1,0,1,0,...,0,1,0,10,2,0,0,0,1,7
7677,10,3,12,3,0,0,0,0,0,0,...,0,0,3,8,2,1,2,0,1,11


In [189]:
base_path = '/home/walml/repos/galaxy-datasets/derived_data/jwst_final/'

for _, galaxy in df.iterrows():
    # if not os.path.isdir(base_path + galaxy['subfolder']):
    #     os.mkdir(base_path + galaxy['subfolder'])
    shutil.copyfile(galaxy['raw_file_loc'], galaxy['file_loc'])

# warning - the pictures seem to be VERY smooth. not sure how useful these will be

In [190]:
df['filename'] = df['raw_file_loc'].apply(os.path.basename)

In [191]:
useful_cols = ['id_str', 'ra', 'dec', 'filename'] + label_cols

In [193]:
train_catalog, test_catalog = train_test_split(df, test_size=0.2, random_state=42)

In [194]:
train_catalog.to_parquet('/home/walml/Dropbox (The University of Manchester)/pytorch-galaxy-datasets/gz_jwst/jwst_train_catalog.parquet', index=False)
test_catalog.to_parquet('/home/walml/Dropbox (The University of Manchester)/pytorch-galaxy-datasets/gz_jwst/jwst_test_catalog.parquet', index=False)

/home/walml/miniforge3/envs/rings/lib/python3.9/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


In [196]:
import hashlib

for loc in [
    '/home/walml/Dropbox (The University of Manchester)/pytorch-galaxy-datasets/gz_jwst/jwst_train_catalog.parquet',
    '/home/walml/Dropbox (The University of Manchester)/pytorch-galaxy-datasets/gz_jwst/jwst_test_catalog.parquet',
    '/home/walml/Dropbox (The University of Manchester)/pytorch-galaxy-datasets/gz_jwst/images.tar.xz'  # compressed from images above
]:
    with open(loc, 'rb') as f:
        md5_checksum = hashlib.md5(f.read()).hexdigest()

    print(md5_checksum)

69e689017e8317abce6b2715382746ae
c23807b06708bbdc5fdaefa284a96094
28ed102a4bcf4bb6693caeebd76a9c44
